In [194]:
import base64
import requests
import pandas as pd
import os
import discogs_client
import time
import numpy as np

In [2]:
# Autorización spotify

In [195]:
client_id = os.environ['spotify_client_id']
client_secret = os.environ['spotify_client_secret']

# función de autenticación de spotify

def spoty_aut (client_id, client_secret):
    url = 'https://accounts.spotify.com/api/token'
    credenciales = client_id + ':' + client_secret
    authorization = "Basic " + base64.b64encode(credenciales.encode()).decode()
    grant_type= 'client_credentials'
    auth = requests.post(url, data={'grant_type':grant_type},
              headers={'Authorization':authorization, 'Content-Type':'application/x-www-form-urlencoded'})
    return auth.json()

auth=spoty_aut(client_id, client_secret)

auth

{'access_token': 'BQDQ5-gPegXeEqQMFb7BR1BA62k8uT9obbNNJCnLM2D2rOggZeoUI8LLAuAwuHOjUli0Who-wYRPPzDfphU',
 'token_type': 'Bearer',
 'expires_in': 3600}

In [197]:
# autorización de la API de discogs
d = discogs_client.Client('ExampleApplication/0.1', user_token="TetTWkQitrxfTKxUfBUaGidepABvNfmADZwDYRfm")

In [196]:
df_set = pd.read_csv("/Users/santiago/ih-bootcamp/proyectos/sherpa/out/skee_mask.csv", index_col=0).reset_index()

In [198]:
df_set

,index,n_track,set_section,artist,track,set_name
0,0,1,1,Anduin,Makepiece In Pieces,2015-03-12 - Skee Mask @ Boiler Room Munich x ...
1,1,2,1,Transllusion,Dirty South Strut,2015-03-12 - Skee Mask @ Boiler Room Munich x ...
2,2,3,1,DJ Gregory,Head Talking,2015-03-12 - Skee Mask @ Boiler Room Munich x ...
3,3,4,1,Kartei Dimension 11,Kartei,2015-03-12 - Skee Mask @ Boiler Room Munich x ...
4,4,5,1,Soundhack,Untitled 2 A2,2015-03-12 - Skee Mask @ Boiler Room Munich x ...
...,...,...,...,...,...,...
197,197,20,1,0lga+Jozef,Untitled,2020-09-30 - Skee Mask - Tsugi Podcast 602
198,198,21,1,Jeff Cochran & Adam Winick,D:EP B2,2020-09-30 - Skee Mask - Tsugi Podcast 602
199,199,22,1,Daniel Jacques,Cast Away,2020-09-30 - Skee Mask - Tsugi Podcast 602
200,200,23,1,Paul Mac,More Over,2020-09-30 - Skee Mask - Tsugi Podcast 602


In [199]:
df_set.drop_duplicates(inplace=True)
df_set.drop('index', axis=1, inplace=True)
df_set.track=df_set.track.str.replace(r"\]", "")

<ipython-input-199-d9b3e90ba65a>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_set.track=df_set.track.str.replace(r"\]", "")


In [167]:
## obteniendo datos de la api de spotify

In [200]:
artists = list(df_set.artist.str.lower().str.replace(" ", "%20"))
tracks = list(df_set.track.str.lower().str.strip().str.replace(" ", "%20"))

In [201]:
lista_df_tracks = []
base = "https://api.spotify.com/v1/"

for i in range(1,len(tracks)):
    
    
    song = requests.get(base+f"search?q=track:{tracks[i]}%20artist:{artists[i]}&type=track",
                    headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
    

    if len(song.content) > 500:
        track = df_set.reset_index()['track'][i]
        track_id = song.json()['tracks']['items'][0]['id']
        album = song.json()['tracks']['items'][0]['album']['name']
        duration = song.json()['tracks']['items'][0]['duration_ms']/60_000
        
        #obteniendo song features
        audio_features = requests.get(base+f"audio-features/{track_id}",
                                  headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
        
        tempo = audio_features.json()['tempo']
        energy = audio_features.json()['energy']
        danceability = audio_features.json()['danceability']
        valence = audio_features.json()['valence']
        key = audio_features.json()['key']
        time_signature = audio_features.json()['time_signature']
        acousticness = audio_features.json()['acousticness']
        instrumentalness = audio_features.json()['instrumentalness']
        mode = audio_features.json()['mode']
        
    else:
        #print(df_set.track[i], df_set.artist[i])
        track = df_set.reset_index()['track'][i]
        track_id = ""
        album = ""
        duration = ""  
    
        tempo = ""
        energy = ""
        danceability = ""
        valence = ""
        key = ""
        time_signature = ""
        acousticness = ""
        instrumentalness = ""
        mode = ""
    #print(album)
    lista_track = [track, track_id, album, duration, tempo, energy, danceability, 
                     valence, key, time_signature, acousticness, instrumentalness, mode]
    lista_df_tracks.append(lista_track)

In [202]:
spoty_df = pd.DataFrame( lista_df_tracks, columns=['track','song_id', 'album', 'duration', 'tempo', 'energy', 'danceability', 'valence'
                                        , 'key', 'time_signature', 'acousticness', 'instruentalness', 'mode'])

In [203]:
spoty_df.head()

,track,song_id,album,duration,tempo,energy,danceability,valence,key,time_signature,acousticness,instruentalness,mode
0,Dirty South Strut,,,,,,,,,,,,
1,Head Talking,28hRZdsfwZ6kCfFCRg1bDv,Kern Exclusives,6.023617,128.969,0.602,0.558,0.473,7,4,0.000927,0.533,1
2,Kartei,,,,,,,,,,,,
3,Untitled 2 A2,,,,,,,,,,,,
4,Untitled 2 A2,,,,,,,,,,,,


In [204]:
spoty_df=spoty_df[spoty_df.song_id!=""]

In [205]:
df_variables=df_set.merge(spoty_df, on="track",how='left')

In [206]:
df_variables.dropna().to_csv("/Users/santiago/ih-bootcamp/proyectos/sherpa/out/spoty/skee_mask_spoty.csv")

In [ ]:
#limpieza nulos

In [31]:
df_variables = pd.read_csv("/Users/santiago/ih-bootcamp/proyectos/sherpa/out/tmp.csv", index_col=0)

In [28]:
set_na = df_variables[['na', 'set_name']].groupby('set_name').sum().reset_index()

In [207]:
df_variables['set_name'].unique()

array(['2015-03-12 - Skee Mask @ Boiler Room Munich x Illian Tape',
       '2016-03-04 - Skee Mask - Groove Podcast 50',
       '2019-08-03 - Skee Mask b2b Zenker Brothers @ Dekmantel Festival, Amsterdam',
       '2019-08-30 - Skee Mask @ Dimensions Festival, Croatia (RA Live)',
       '2019-12-19 - Skee Mask - Ilian Tape Podcast Series 046',
       '2020-09-30 - Skee Mask - Tsugi Podcast 602'], dtype=object)

In [30]:
df_variables[['na', 'set_name']].groupby('set_name').sum()

,na
set_name,
2011-08-23 - Objekt @ Boiler Room Berlin 001,5
2011-11-14 - Objekt - FACT Mix 300,8
"2012-03 - Rob Booth, Objekt - Electronic Explorations 190",6
2015-03-12 - Skee Mask @ Boiler Room Munich x Illian Tape,22
2016-03-04 - Skee Mask - Groove Podcast 50,16
"2019-08-03 - Skee Mask b2b Zenker Brothers @ Dekmantel Festival, Amsterdam",10
"2019-08-30 - Skee Mask @ Dimensions Festival, Croatia (RA Live)",17
2019-12-19 - Skee Mask - Ilian Tape Podcast Series 046,19
2020-09-30 - Skee Mask - Tsugi Podcast 602,30


In [124]:
df_variables.merge(set_na, on='set_name', how='left')['na_y']

0      14
1      14
2      14
3      14
4      14
       ..
722     3
723     3
724     3
725     3
726     3
Name: na_y, Length: 727, dtype: int64

In [125]:
df_variables=df_variables[df_variables.merge(set_na, on='set_name', how='left')['na_y']<8]

In [126]:
df_variables[['na', 'set_name']].groupby('set_name').sum().reset_index()

,set_name,na
0,2012-02-08 - Bonobo @ Boiler Room 77 - Ninja T...,1
1,"2013-12-08 - Bonobo - Liquid Radio, FM4",2
2,"2014-04-06 - Bonobo (Live PA) - Fireside Chat,...",4
3,2014-04-12 - Bonobo - Essential Mix,6
4,2014-09-02 - Bonobo - Mixed By,4
5,2014-11-20 - Bonobo (Live PA) @ Boiler Room Lo...,7
6,"2015-12-18 - Bonobo - Outlier Radio 001, NTS R...",5
7,2016-04-07 - George FitzGerald b2b Bonobo - Re...,7
8,"2016-12-16 - Bonobo - Outlier Radio 004, NTS R...",7
9,"2017-02-10 - Bonobo @ Mixmag Live, Village Und...",6


In [127]:
df_variables.isnull().sum()

n_track             0
set_section         0
artist              0
track               0
set_name            0
song_id            57
album              57
duration           57
tempo              57
energy             57
danceability       57
valence            57
key                57
time_signature     57
acousticness       57
instruentalness    57
mode               57
na                  0
dtype: int64

In [128]:
df_variables

,n_track,set_section,artist,track,set_name,song_id,album,duration,tempo,energy,danceability,valence,key,time_signature,acousticness,instruentalness,mode,na
125,1,1,Bonobo,Cirrus,2012-02-08 - Bonobo @ Boiler Room 77 - Ninja T...,2lJ4d8MCT6ZlDRHKJ1br14,The North Borders,5.870767,119.042,0.844,0.64,0.364,2,4,0.395,0.933,0,False
126,2,1,Eliphino,More Than Me,2012-02-08 - Bonobo @ Boiler Room 77 - Ninja T...,20ieXcjQBv4ePXkWxhvKPV,More Than Me,7.019933,117.994,0.714,0.851,0.414,9,4,0.00383,0.38,0,False
127,3,1,FaltyDL,Straight And Arrow,2012-02-08 - Bonobo @ Boiler Room 77 - Ninja T...,7G2ws9VEzcZk51nnp4xuYu,Straight & Arrow,3.464333,130.038,0.723,0.791,0.132,8,4,0.00106,0.767,1,False
128,4,1,Rocketnumbernine,Matthew And Toby (Four Tet Remix),2012-02-08 - Bonobo @ Boiler Room 77 - Ninja T...,6KuF6uqMIvaafrF1LCWcEg,Dubfire - A Transmission,9.784,125.007,0.483,0.744,0.281,7,4,0.572,0.94,1,False
129,5,1,Bonobo,Sapphire,2012-02-08 - Bonobo @ Boiler Room 77 - Ninja T...,1d5hX0RYHcDM4aHJHSm5cr,The North Borders,4.795467,135.024,0.631,0.641,0.0549,0,4,0.048,0.612,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,22,1,Djrum,Showreel Pt. 2,"2017-11-20 - Bonobo - Outlier Radio, NTS Radio",4Tg0ra2NwFahHXiboKtfPW,"Showreel, Pt. 2",9.023783,92.635,0.787,0.274,0.143,8,3,0.06,0.689,1,False
723,23,1,Appleblim,Sacred Rowan Grove,"2017-11-20 - Bonobo - Outlier Radio, NTS Radio",34bMRysVmX1E2nBu9YjAsh,Displaced Soundtracks,5.373217,120.047,0.905,0.423,0.352,8,4,0.089,0.694,1,False
724,24,1,Agnes Obel,Stretch Your Eyes (Quiet Village Remix),"2017-11-20 - Bonobo - Outlier Radio, NTS Radio",06xHJ8x3p0Kaw1zYfevxSA,Stretch Your Eyes (Quiet Village Remix),9.43655,120.004,0.413,0.579,0.0386,6,4,0.436,0.899,0,False
725,25,1,Penya,Why So Angry,"2017-11-20 - Bonobo - Outlier Radio, NTS Radio",7FLWpgPAF99CZvGcNBGwGU,Super Liminal,4.195517,125.019,0.726,0.7,0.21,6,4,0.195,0.693,0,False


In [129]:
df_variables.to_csv("/Users/santiago/ih-bootcamp/proyectos/sherpa/out/bonobo_spoty.csv")